In [1]:
import argparse
import os

from utils.utils import *
from scripts.config import DefaultArgsNamespace
import torch
import torch.nn as nn
import torchvision.models as models
from datautils.ems import *
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import wandb
from datetime import datetime

from ego_rate_estimator.cpr_rate_detection_video import ego_rate_detect

import cv2

from PIL import Image

import matplotlib.pyplot as plt

import argparse
import warnings
warnings.filterwarnings("ignore", message="Accurate seek is not implemented for pyav backend")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



/sfs/gpfs/tardis/home/cjh9fw/Desktop/2024/repos/EgoExoEMS/Benchmarks/CPR_quality/late_fusion


ImportError: cannot import name 'tools' from 'tools.tools' (/sfs/gpfs/tardis/home/cjh9fw/Desktop/2024/repos/EgoExoEMS/Benchmarks/CPR_quality/late_fusion/tools/tools.py)

In [ ]:
# get cmd line args
args = DefaultArgsNamespace()

keysteps = args.dataloader_params['keysteps']
out_classes = len(keysteps)

modality = args.dataloader_params['modality']
print("Modality: ", modality)
print("Num of classes: ", out_classes)

window = args.dataloader_params['observation_window']
print("Window: ", window)

task = args.dataloader_params['task']
print("Task: ", task)


# train_loader, val_loader, test_loader = get_dataloaders(args)
train_loader, val_loader, test_loader, train_class_stats, val_class_stats = eee_get_dataloaders(args)
args.dataloader_params['train_class_stats'] = train_class_stats
args.dataloader_params['val_class_stats'] = val_class_stats

Modality:  ['video', 'smartwatch', 'depth_sensor']
Num of classes:  35
Window:  150
Task:  cpr_quality
********** ========== **********
Loading dataloader for CPR quality task
video_path: /standard/UVA-DSA/NIST EMS Project Data/EgoExoEMS_CVPR2025/Dataset/Final/ms1/cardiac_arrest/3/GoPro/GX010394_encoded_trimmed.mp4
audio_path: None
flow_path: None
rgb_path: None
resnet_path: None
resnet_exo_path: None
smartwatch_path: /standard/UVA-DSA/NIST EMS Project Data/EgoExoEMS_CVPR2025/Dataset/Final/ms1/cardiac_arrest/3/smartwatch_data/sync_smartwatch.csv
depth_sensor_path: /standard/UVA-DSA/NIST EMS Project Data/EgoExoEMS_CVPR2025/Dataset/Final/ms1/cardiac_arrest/3/distance_sensor_data/sync_depth_sensor.csv
data_types: ['video', 'smartwatch', 'depth_sensor']
video_path: /standard/UVA-DSA/NIST EMS Project Data/EgoExoEMS_CVPR2025/Dataset/Final/ms1/chest_pain/6/GoPro/GX010408_encoded_trimmed.mp4
audio_path: None
flow_path: None
rgb_path: None
resnet_path: None
resnet_exo_path: None
smartwatch_path

## Functions for rate detection

In [ ]:
def detect_rate(frames,smartwatch,gt,window_size):
    # iterate through the frames using a tumbling window with the given window size
    # and calculate the detection rate
    # for each window

    for i in range(0, len(frames), window_size):
        # get the current window
        window = frames[i:i+window_size]
        # get the current smartwatch data
        smartwatch_window = smartwatch[i:i+window_size]
        # get the current gt data
        gt_window = gt[i:i+window_size]

        print("Window: ", window.shape)
        print("Smartwatch: ", smartwatch_window.shape)
        print("GT: ", gt_window.shape)


        # call video rate detection function
        ego_cpr_rate = ego_rate_detect(window)

    return None,None,None,None


    
    

In [ ]:
for i, batch in enumerate(train_loader):
    print("****************")
    print("Batch: ", i)
    input,feature_size, gt_sensor_data = preprocess(batch, modality, None, device, task=task)
    print("Subject: ", batch['subject_id'])
    print("Trial", batch['trial_id'])
    print("Input frames: ", input['frames'].shape)

    fused_rate, sw_rate, video_rate, gt_rates = detect_rate(input['frames'][0], input['smartwatch'][0], gt_sensor_data[0], window)

    break
    
    # # sample image
    # image = input['frames'][0][0].permute(1,2,0).cpu().numpy()
    # # convert to PIL image
    # image = Image.fromarray(image)
    # # show pil image
    # plt.imshow(image)
    
    # print("Input smartwatch: ", input['smartwatch'].shape)
    # print("gt_sensor_data shape: ", gt_sensor_data.shape)
    
    # # plot the smartwatch data (3-axis accelerometer)
    # smartwatch_data = input['smartwatch'][0].cpu().numpy()
    # plt.figure(figsize=(10, 5))
    # plt.plot(smartwatch_data[:, 0], label='X-axis')
    # plt.plot(smartwatch_data[:, 1], label='Y-axis')
    # plt.plot(smartwatch_data[:, 2], label='Z-axis')
    # plt.title('Smartwatch Data')
    # plt.xlabel('Time')
    # plt.ylabel('Acceleration')
    # plt.legend()
    # plt.show()
    
    # # plot the gt_sensor_data (1-axis distance)
    # gt_sensor_data = gt_sensor_data[0].cpu().numpy()
    # plt.figure(figsize=(10, 5))
    # plt.plot(gt_sensor_data, label='Distance')
    # plt.title('Ground Truth Sensor Data')
    # plt.xlabel('Time')
    # plt.ylabel('Distance')
    # plt.legend()
    # plt.show()
    # print("****************")
    
    # break
    

****************
Batch:  0
Special case for CPR quality task
Subject:  ['ng3']
Trial ['6']
Input frames:  torch.Size([1, 456, 3, 224, 224])
Window:  torch.Size([150, 3, 224, 224])
Smartwatch:  torch.Size([150, 3])
GT:  torch.Size([150, 1])
Window:  torch.Size([150, 3, 224, 224])
Smartwatch:  torch.Size([150, 3])
GT:  torch.Size([150, 1])
Window:  torch.Size([150, 3, 224, 224])
Smartwatch:  torch.Size([150, 3])
GT:  torch.Size([150, 1])
Window:  torch.Size([6, 3, 224, 224])
Smartwatch:  torch.Size([6, 3])
GT:  torch.Size([6, 1])
